In [1]:
import sys   
sys.path.append('../../src/features')
import build_features as feature_builder

In [5]:
from datetime import date

start_date = date(2017, 1, 1)
end_date = date(2017, 12, 31)
airport_code = 'ATL'
path_to_db = "sqlite:///../../data/processed/airlines.db"


features = feature_builder.get_features_for_delay_classification(start_date, end_date, airport_code, path_to_db)

In [6]:
features.head()

,departure_was_delayed_15,origin,dest,carrier,departure_time_scheduled,distance,elapsed_time_scheduled,departure_time,departure_month,departure_date,...,departure_hod,airport,hourly_visibility,hourly_dry_bulb_temp_f,hourly_precipitation,hourly_wind_speed,hourly_wind_gust_speed,hourly_station_pressure,measurement_hour,m_hour_shifted


In [7]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 21 columns):
departure_was_delayed_15    0 non-null int64
origin                      0 non-null object
dest                        0 non-null object
carrier                     0 non-null object
departure_time_scheduled    0 non-null object
distance                    0 non-null int64
elapsed_time_scheduled      0 non-null int64
departure_time              0 non-null datetime64[ns]
departure_month             0 non-null int64
departure_date              0 non-null int64
departure_dow               0 non-null int64
departure_hod               0 non-null int64
airport                     0 non-null object
hourly_visibility           0 non-null object
hourly_dry_bulb_temp_f      0 non-null object
hourly_precipitation        0 non-null object
hourly_wind_speed           0 non-null object
hourly_wind_gust_speed      0 non-null object
hourly_station_pressure     0 non-null object
measurement_hour            0 non-nul